# Memory Virtualization

The OS virtualizes memory, that is, it gives the illusion that every process has full access to the system's memory.

Every process has a memory address space starting at byte 0 and going up to some arbitrarily large byte number.

A computer has physical memory (its RAM) on the order of 4-64 GB.

The physical memory to us is a sequence of bytes going from byte 0 to the maximum byte of the machine.

For example, if our machine has 32KB of memory, physical memory consists of bytes 0-32767

While there is only a single byte 0 on the machine, every process thinks it has access to its own byte 0.

In order to give this illusion, the OS maps every process' **Virtual Address Space (VAS)** onto some section of physical memory.

<br>
<img src="images/01-process-image.png" width="200">
<br>

The OS can decide to map this 16 KB VAS to somewhere in physical memory.

<br>
<img src="images/02-physical-mem.png" width="500">
<br>

## Base & Bounds

At process creation, the OS allotes the entire address space, storing it somewhere in physical memory.

The OS remembers where the virtual address space starts in physical memory (the **base**) and how large the VAS is (the **bounds**).

## Address Translation

Whenever a process attempts to access a memory location (a virtual address), the OS must translate from the virtual address to the physical address where that virtual address is actually stored and access the value at that physical location.

This translation is hidden from user processes.

The address translation is simple:

`physical_address = base + virtual_address`

The OS stores the `base` and `bounds` for every process.

As a process is running, these two values are stored in registers on the CPU (that only the OS can access).

Every single address access triggers a hardware translation for the OS the calculate where the virtual address is stored in physical memory.

The part of the CPU that stores these registers and the circuitry to perform the translations is known as the **Memory Management Unit (MMU)** on the CPU.

For security, every process is only allowed to write within its bounds. If the process tries to write outside of its bounds the OS performs  exception handling. For base and bounds, it simply raises a fault and kills the process.

### Example Translations

4KB Virtual Address Space, starting in physical memory at 20KB.

**Base:** 20KB aka byte 20480

**Bounds:** 4KB - 1: 4095

The bounds indicates the last writable virtual address by the process.

This allows for a simple process to check if the virtual address is out of bounds:

```c
if(VA > BOUNDS) {
    raise(BASE_AND_BOUNDS_FAULT)
}
```

```
VA    ->  Physical Address
------------------------------
0     ->  20KB
1KB   ->  21KB
2000  ->  22480
4400  ->  FAULT, out of bounds
```